In [169]:
import pandas as pd
import time
import numpy as np
import funcoes
from datetime import datetime, timedelta
from dateutil import parser
from flask import Flask
from flask_mail import Mail, Message
import os

# Configs
pd.set_option("display.date_dayfirst", True)
# Variaveis globais
tks = ['HihrPwM6iplpMc20YafyI0X20hYBwMJjn9O5b3uRXLc%3D','NHvW6AeyLM3HFKp1HwXQIbDbGVdDOyMhvbyHgEoZY57S4pEFqdDDKy1R%2FVQHplHm','UVzI3W1fv8mAK28sWaTc%2BOlAefhF1q26WWTfaOmb%2BSY6F1b2IE43cgRMrafYMySh','oAlfna7l2QNiS6nZo4oIQSQCpNo5XX9iydYqEGfMVkDVz8fBG2Vl6bW9lChTtGtP', 'vcMSbDWqyMNu8fcVLol8l0dZ0DZhinIFKyydHCBSAb%2B5RloWjteXNY9345MIcbUQ']
metodo = 'GET'
url = "https://elysia.zeev.it"
tipoUrl = "/api/2/assignments"
filtro = {'flowId': 0}

# Datas
today = datetime.today().strftime('%Y-%m-%d')
periodo1 = datetime.today() - timedelta(days=7)
periodo1 = periodo1.strftime('%Y-%m-%d')
periodo2 = today 
print("No período de {0} e {1}".format(str(periodo1), str(periodo2)))

No período de 2022-11-01 e 2022-11-08


In [70]:
# Se a API alterar seu campos novamente, sera necessario alterar apenas alguns filtros desta funcao
def request_Orquestra(method='GET',urlReq="https://elysia.zeev.it",typeReq="/api/2/assignments",tokens={},filters={}, forceRequisition:bool=False) -> list:
        try:
                # Variaveis de controle
                row = 1
                i = 1
                retorno = []
                # filtros base para o payload
                if not typeReq.lower().__contains__('report'):
                        filters['recordsPerPage'] = 100
                        seconds = 1
                else:
                        if not forceRequisition: 
                                print("---- >> Requisicao lenta... << ---- ")
                                filters['recordsPerPage'] = 30
                                seconds = 3
                        else:
                                print("---- >> Requisicao forçada... << ---- ")
                                filters['recordsPerPage'] = 100
                                seconds = 1    
                
                filters['mobileEnabledOnly'] = False
                # Faz a busca para cada token
                for tk in tokens:
                        # Reseta variaveis de controle
                        row = 1
                        i = 1
                        # Iteracao com token
                        authorization= {'Authorization': "Bearer " + tk}
                        while row != 0:
                                # Aguardo x segundos devido ao limite de requisicoes - lembrando que a limite de requisicoes por segundos, minutos, horas, dias e meses
                                time.sleep(seconds) 
                                
                                # Incremento de pagina
                                filters['pageNumber'] = i
                                
                                # Requisicao
                                getReq = funcoes.requests_API_Orquestra(metodo=method, 
                                                                        urlAcesso=urlReq, 
                                                                        tipoAcesso=typeReq, 
                                                                        head=authorization, 
                                                                        payload=filters)
                                
                                # Resultado JSON da requisicao
                                result = getReq.json()
                                        
                                # Checa se esta vazio o resultado, o que significa que nao tem mais paginas
                                if (len(result) == 0) or (getReq.status_code != 200):
                                        break
                                else:
                                        # coloca cada retorno na lista
                                        retorno.append(result)
                                        i+=1
                # Cada usuario do token fica em uma sublista da lista de retorno, por isto a necessidade de 'explodir' ela dentro de outra lista.
                retornoLista = [item for sublista in retorno for item in sublista]
                return retornoLista 
        except Exception as e:
                funcoes.printError(e)
                return [] 

In [56]:
def formata_DataFrame(lista:list):
    '''
        Funcao
        ------
            Faz a formatacao do data frame colocando as no formato correto e criando uma coluna chamada `atrasada`.

        Parametros
        ----------
            lista: list (obrigatorio)
                Lista com dicionario JSON, para formatacao em data frame.
        
        Retorno
        -------
            Retorna o dicionario em um formato de `DataFrame`.
            * Se erro, retorna -1. 
    '''
    # verifica se houve retorno
    try:
        # ocorreu erro ou nao ocorreram retornos - provisorio
        if len(lista) <= 0:
            df = pd.DataFrame
            return df
        # houve retorno de linhas
        elif len(lista) > 0:
            df = pd.DataFrame(lista)
            # Colocando o idTask para conferencia
            df['idTask'] = pd.DataFrame(df['instance'].apply(pd.Series))['id']
            # selecionando apenas as colunas necessarias
            df = df[['idTask','id','taskName','requestName','startDateTime','expirationDateTime','flow','instance']]

            # convertendo colunas para datetime
            df['expirationDateTime'] = pd.to_datetime(df['expirationDateTime'])
            df['startDateTime'] = pd.to_datetime(df['startDateTime'])

            # formatando data em DD/MM/YY
            df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%Y-%m-%d')
            df.loc[:,'startDateTime'] = df.loc[:,'startDateTime'].dt.strftime('%Y-%m-%d')

            # convertendo colunas para datetime
            df['expirationDateTime'] = pd.to_datetime(df['expirationDateTime'])
            df['startDateTime'] = pd.to_datetime(df['startDateTime'])

            # adiciona coluna de tarefa atrasada
            df['taskAtrasada'] = False
            df.loc[today > df['expirationDateTime'],['taskAtrasada']] = True
            
            return df
    except Exception as e:
        print('--> Erro na funcao \'contagem_CS1\' <--')
        funcoes.printError(e)
        return -1

In [94]:
def contagem_Instancias_Orquestra(nomeSolicitacao:str, nomeTarefa:str,per1:datetime,per2:datetime,dataFrame:pd.DataFrame, status:str='endDateTime') -> int:
    '''
     Funcao
     ------
            Retorna a contagem de tarefas fechadas ou abertas para determinada solicitacao em um determinado periodo.

    Parameters
    ----------
        nomeSolicitacao: str (obrigatorio)
            Nome da solicitacao que sera buscada e contada a instancia.
        
        nomeTarefa: str (obrigatorio)
            Nome da tarefa para ser contada a sua quantidade.

        dataFrame: dataFrame Pandas (obrigatorio)
            Dataframe que sera utilizado como base de dados para a contagem.
        
        per1: datetime (obrigatorio)
            Periodo inicial para busca.

        per2: datetime (obrigatorio)
            Periodo final para busca.

        status: str (opcional)
            Status da data para verificar o periodo - podendo ser `endDateTime` ou `startDateTime`.
    Retorno
    -------
        int: Retorna a quantidade de instancias ja finalizadas desta tarefa.
        * Se erro, retorna -1   
    '''
    try:
        # Formatar parametros - remove espacos em branco, deixar em letra minuscula
        nomeSolicitacao = str(nomeSolicitacao).lower().replace(" ","")
        nomeTarefa = str(nomeTarefa).lower()
        # Projetos 
        projetos = dataFrame.loc[(dataFrame['requestName'].str.lower().str.replace(" ","").str.contains(nomeSolicitacao))]
        # Contagem
        cont=0
        # Percorre linha a linha do dataframe, verificando cada instancia aberta ou fechada da solicitacao 
        for row in projetos.values:
            for instance in row[19]:
                task = str(instance['task']['name']) 
                date = parser.parse(instance[status]).strftime("%Y-%m-%d")
                if task.lower().__contains__(nomeTarefa):
                    if date >= per1 and date <= per2:
                        cont+=1
                        break
        return cont
    except Exception as e:
        print('--> Erro na funcao \'contagem_Instancias_Orquestra\' <--')
        funcoes.printError(e)
        return -1

def instances_Report_Orquestra(userToken:str, showFinishedTasks:bool=True, activeInstances:bool=True, flowId:str=''):
    '''
        Funcao
        ------
            Lista todas instâncias de solicitações que a pessoa relacionada ao token possui permissão de consultar de acordo com filtros.
        
        Parametros
        ----------
            userToken: str (obrigatorio)
                Token do usuario orquestra para solicitar a requisicao.

            showFinishedTasks: boolean (opcional)
                Definir se vai filtrar por instancias fechadas ou instancias abertas das solicitacoes.
            
            activeInstances: boolean (opcional)
                Definir se a solicitacao esta `Finalizada` ou `Em andamento`.

            flowId: str (opcional)
                Filtrar pelo id de instancia.
        Retorno 
        -------
            Retorna um DataFrame(Pandas).
            * Se erro, retorna -1.
    '''
    try:
        # Filtro padrao - Solicitacoes Em andamento e com instancias Finalizadas
        f = {"startDateIntervalBegin": "2000-01-01T00:00:00",
                "startDateIntervalEnd": "2030-12-31T23:59:59",
                "showFinishedInstanceTasks": True, 
                "showPendingInstanceTasks": False,
                "active": activeInstances}

        # Escolhe se vai mostrar solicitacoes Finalizadas ou Em andamento
        if not activeInstances:
            # Vai mostrar solicitacoes Finalizadas
            f['showFinishedInstanceTasks'] = False
            f['showPendingInstanceTasks'] = False
        else:
            # Vai mostrar solicitacoes Em andamento e com instancias Pendentes
            if showFinishedTasks == False: 
                f['showFinishedInstanceTasks'] = False
                f['showPendingInstanceTasks'] = True

        # Filtro de ID para solicitacoes especificas
        if flowId != '': 
            f['flowId'] = flowId
        # Requisicao 
        lista = request_Orquestra(method='GET',
                                    urlReq=url,
                                    typeReq='/api/2/instances/report',
                                    tokens=[userToken],
                                    filters=f,
                                    forceRequisition=(not activeInstances)) # Requisicao forcada no caso de solicitacoes Finalizadas
        df = pd.DataFrame(lista)
        return df
    except Exception as e:
        print('--> Erro na funcao \'instances_Report_Orquestra\' <--')
        funcoes.printError(e)
        return -1

In [131]:
def contagem_CS1(dataFrame:pd.DataFrame) -> list:
    '''
        Funcao
        ------
            Faz a contagem de clientes no fluxo CS1.

        Parametros
        ----------
            dataFrame: DataFrame (obrigatorio)
                Dataframe a ser utilizado na contagem.
        
        Retorno
        -------
            Retorna uma lista com os 3 valores de contagem para CS1: `[contCs1Abertos,contCs1Efetivos,contCS1Atrasadas]`.
            * Se erro, retorna uma lista de -1.
    '''
    try:
        # CS1 Abertos sao os Efetivos Em espera + Em Aberto nas atribuicoes de cada usuario -> Pegar o relogio de 7/15 dias
        df = dataFrame
        cs1 = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) & ((df['taskName'].str.lower().str.contains('ligação do cs')) | (df['taskName'].str.lower().str.replace(" ","").str.contains('cooldown')))]
        contCs1Abertos = len(cs1)

        # CS1 Efetivos sao os Em espera do 'Servico Orquestra'
        # Entrou no relogio de 7 ou 15 dias na semana do script, conta como efetivo
        cs1Efetivos = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) 
                            & (df['taskName'].str.lower().str.replace(" ","").str.contains('cooldown'))
                            & (df['startDateTime'].between(periodo1,periodo2, inclusive='both'))]
        contCs1Efetivos = len(cs1Efetivos)

        # CS1 Atrasados 
        contCS1Atrasadas = len(cs1.loc[(df['taskAtrasada'] == True) & (df['taskName'].str.lower().str.contains('ligação do cs'))])
        return [contCs1Abertos,contCs1Efetivos,contCS1Atrasadas]
    except Exception as e:
        print('--> Erro na funcao \'contagem_CS1\' <--')
        funcoes.printError(e)
        return [-1]

def contagem_CS2_Abertos_Degustacao(instancesDataFrame:pd.DataFrame) -> int:
    try:
        # Formatar parametros - remove espacos em branco, deixar em letra minuscula
        nomeSolicitacao = str('degustação').lower().replace(" ","")
        nomeTarefa = str('ligação do cs').lower()
        # Projetos 
        projetos = instancesDataFrame.loc[(instancesDataFrame['requestName'].str.lower().str.replace(" ","").str.contains(nomeSolicitacao))]
        # Contagem
        cont=0
        aberto=0
        # Percorre linha a linha do dataframe, verificando cada instancia aberta ou fechada da solicitacao 
        for row in projetos.values:
            for instance in row[19]:
                task = str(instance['task']['name']) 
                if task.lower().__contains__(nomeTarefa):
                    cont+=1
            if cont>=1:
                aberto+=1 # conta a quantidade de abertos em degustacao
                cont=0
        return aberto
    except Exception as e:
        print('--> Erro na funcao \'contagem_CS2_Degustacao\' <--')
        funcoes.printError(e)
        return -1

def contagem_CS2_Efetivo_Degustacao(instancesReportClose:pd.DataFrame) -> list:
    try:
        # verifica se o dataframe nao esta vazio
        if not instancesReportClose.empty:
            # Pega apenas as colunas necessarias
            solicitationsCloseDF = instancesReportClose[['id','requestName','active','endDateTime']]
            # Formata a data
            solicitationsCloseDF['endDateTime'] = pd.to_datetime(solicitationsCloseDF['endDateTime'])
            solicitationsCloseDF['endDateTime'] = solicitationsCloseDF['endDateTime'].dt.strftime('%Y-%m-%d')
            solicitationsCloseDF['endDateTime'] = pd.to_datetime(solicitationsCloseDF['endDateTime'])
            # Remove linhas sem data de finalizacao
            solicitationsCloseDF.dropna(inplace=True)
            # Conta a quantidade de CS2 Degustacao efetivo, ou seja, que tenha finalizado a ultima ligacao...
            return [len(solicitationsCloseDF.loc[solicitationsCloseDF['endDateTime'].between(periodo1, periodo2, inclusive="both")])]
        else:
            print('---- >> DataFrame vazio! << ----')
            return [0]
    except Exception as e:
        print('--> Erro na funcao \'contagem_CS2_Degustacao\' <--')
        funcoes.printError(e)
        return [-1]

def contagem_CS2(dataFrame:pd.DataFrame, instancesDataFrame:pd.DataFrame, instancesCloseDataFrame:pd.DataFrame) -> list:
    '''
        Funcao
        ------
            Faz a contagem de clientes no fluxo CS2.
        
        Paramentros
        -----------
            dataFrame: DataFrame (obrigatorio)
                DataFrame utilizado para a contagem.
            
            instancesDataFrame: DataFrame (obrigatorio)
                DataFrame utilizado para a contagem.
        
        Retorno
        -------
            Retorna uma lista com os 3 valores de contagem para o CS2: `[contCs2Abertos,contCs2Efetivos,contCs2Atrasados]`
            * Se erro, retorna uma lista de -1.
    '''
    try:
        df = dataFrame
        # CS2 Em aberto -> Para degustacao a etapa 'Inicio do mes seguinte' contabiliza os em aberto apos email = acomp personalizado 1/3
        cs2 = df.loc[(df['requestName'].str.lower().str.contains('degustação')) | (df['requestName'].str.lower().str.contains('normal')) & ~(df['taskName'].str.lower().str.contains('relatório')) | (df['taskName'].str.lower().str.contains('suporte'))]
        cs2Degustacao = contagem_CS2_Abertos_Degustacao(instancesDataFrame)
        cs2Normal = contagem_fluxo_personalizado(instancesDataFrame)[0]
        contCs2Abertos =  cs2Degustacao + cs2Normal
        
        # CS2 Efetivos -> Em degustacao -> Necessario contabilizar apenas a quarta ligacao como efetivo
        cs2NormalInstancias = instancesDataFrame.loc[instancesDataFrame['requestName'].str.lower().str.contains('normal')]
        cs2DegustacaoEfetivo = contagem_CS2_Efetivo_Degustacao(instancesCloseDataFrame)[0]
        cs2NormalEfetivo = contagem_Instancias_Orquestra('normal', 'ligação do cs', '2020-01-01', periodo2, cs2NormalInstancias)
        contCs2Efetivos =  cs2DegustacaoEfetivo + cs2NormalEfetivo
        
        # CS2 Atrasados
        cs2Atrasados = cs2.loc[(cs2['taskName'].str.lower().str.contains('ligação do cs')) & (cs2['taskAtrasada'] == True)]
        contCs2Atrasados = len(cs2Atrasados)
        return [contCs2Abertos,contCs2Efetivos,contCs2Atrasados]
    except Exception as e:

        funcoes.printError(e)
        return [-1]

def contagem_fluxo_personalizado(instancesDataFrame:pd.DataFrame) -> list:
    '''
        Funcao
        ------
           Faz a contagem de clientes no fluxo personalizado.
        
        Parametros
        ----------
            instancesDataFrame: DataFrame (obrigatorio)
                DataFrame utilizado para contagem.
        
        Retorno
        -------
            Retorna uma lista contendo a quantidade de clientes para o fluxo personalizado: `[contClientesfluxoNormal]`
            * Se erro, retorna uma lista de -1.
    '''
    try:
        df = instancesDataFrame
        contClientesfluxoNormal = len(df.loc[df['requestName'].str.lower().str.contains("normal")]) 
         # Retorno com a contagem
        return [contClientesfluxoNormal]
    except Exception as e:
        print('--> Erro na funcao \'fluxo_personalizado\' <--')
        funcoes.printError(e)
        return [-1]

def contagem_layouts_documentos(dataFrame:pd.DataFrame,instancesDataFrame:pd.DataFrame) -> list:
    '''
        Funcao
        ------
           Faz a contagem de documentos e layouts a enviar, enviados e com envio atrasado.
        
        Parametros
        ----------
            dataFrame: DataFrame (obrigatorio)
                DataFrame utilizado para contagem.

            instancesDataFrame: DataFrame (obrigatorio)
                DataFrame utilizado para contagem.
        
        Retorno
        -------
            Retorna uma lista contendo uma sublista para documentos e layouts a quantidade para cada contagem: `[[contEnvLayoutPendentes,contEnvLayoutRealizados,contEnvLayoutAtrasadas],[contEnvDocPendentes,contEnvDocRealizados,contEnvDocAtrasados]]`
            * Se erro, retorna uma lista de -1.
    '''
    try:
        df = dataFrame
        # Envio de Layout Pendentes 
        envLayout = df.loc[(df['requestName'] == 'Projeto  v. 20') & (df['taskName'] == 'Baixar Layout e apresentar ao cliente')]
        contEnvLayoutPendentes = len(envLayout)
        # Envio de Layout Realizados - Contagem por Períodos
        contEnvLayoutRealizados = contagem_Instancias_Orquestra('projeto v.20', 'baixar layout', periodo1, periodo2, instancesDataFrame)
        # Envio de Layout Atrasados
        contEnvLayoutAtrasadas = len(envLayout[(envLayout['taskName'] == 'Baixar Layout e apresentar ao cliente') & (envLayout['taskAtrasada'] == True)])

        # Envio de Documentos Pendentes
        envDoc = df.loc[(df['requestName'] == 'Projeto  v. 20') & (df['taskName'] == 'Entregar Documentação ao Cliente')]
        contEnvDocPendentes = len(envDoc)
        # Envio de Documentos Realizados - Contagem por Períodos
        contEnvDocRealizados = contagem_Instancias_Orquestra('projeto v.20','entregar documentação', periodo1, periodo2, instancesDataFrame)
        # Envio de Documentos Atrasados 
        contEnvDocAtrasados = len(envDoc.loc[envDoc['taskAtrasada'] == True])

        return [contEnvLayoutPendentes,contEnvLayoutRealizados,contEnvLayoutAtrasadas],[contEnvDocPendentes,contEnvDocRealizados,contEnvDocAtrasados]
    except Exception as e:
        print('--> Erro na funcao \'contagem_layouts_documentos\' <--')
        funcoes.printError(e)
        return [-1,-1,-1],[-1,-1,-1]


def cria_DataFrame_OKRs() -> None:
    # Requisicao com dados de atribuicoes aos usuarios
    lista = request_Orquestra(method=metodo,urlReq=url,typeReq=tipoUrl,tokens=tks)
    df = formata_DataFrame(lista)

    # Requisicao com dados de instancias
    instancesReport = instances_Report_Orquestra('enb4iHROSDegvkiZv17Fxw%2BkQ7q7Zidu1PKalcpC4o4%3D')
    instancesReportClose = instances_Report_Orquestra('enb4iHROSDegvkiZv17Fxw%2BkQ7q7Zidu1PKalcpC4o4%3D',activeInstances=False,flowId='255')

    layouts, docs = contagem_layouts_documentos(df, instancesReport)
    contagens = contagem_CS1(df) + contagem_CS2(df,instancesReport, instancesReportClose) + contagem_fluxo_personalizado(instancesReport) + docs + layouts
    listIndex =['CONTATOS EM ABERTO - CS1', 'CONTATOS EFETIVOS - CS1', 'CONTATOS ATRASADOS - CS1',
            'CONTATOS EM ABERTO - CS2', 'CONTATOS EFETIVOS - CS2', 'CONTATOS ATRASADOS - CS2',
            'Nº DE CLIENTES NO ACOMPANHAMENTO PERSONALIZADO',
            'ENVIOS DE DOCUMENTOS PENDENTES', 'ENVIOS DE DOCUMENTOS REALIZADOS', 'ENVIOS DE DOCUMENTOS ATRASADOS',
            'ENVIOS DE LAYOUT PENDENTE', 'ENVIOS DE LAYOUT REALIZADOS', 'ENVIOS DE LAYOUT ATRASADOS']
    contagemDF = pd.DataFrame(contagens, index=listIndex,columns=['Contagem'])
    contagemDF

In [90]:
# Requisicao com dados de atribuicoes aos usuarios
lista = request_Orquestra(method=metodo,urlReq=url,typeReq=tipoUrl,tokens=tks)
df = formata_DataFrame(lista)

# Requisicao com dados de instancias
instancesReport = instances_Report_Orquestra('enb4iHROSDegvkiZv17Fxw%2BkQ7q7Zidu1PKalcpC4o4%3D')
instancesReportClose = instances_Report_Orquestra('enb4iHROSDegvkiZv17Fxw%2BkQ7q7Zidu1PKalcpC4o4%3D',activeInstances=False,flowId='255')

C:\Users\Lukas\AppData\Local\Temp\ipykernel_16016\1088200803.py:36: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%Y-%m-%d')
C:\Users\Lukas\AppData\Local\Temp\ipykernel_16016\1088200803.py:37: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'startDateTime'] = df.loc[:,'startDateTime'].dt.strftime('%Y-%m-%d')


False
{'startDateIntervalBegin': '2000-01-01T00:00:00', 'startDateIntervalEnd': '2030-12-31T23:59:59', 'showFinishedInstanceTasks': True, 'showPendingInstanceTasks': False, 'active': True}
---- >> Requisicao lenta... << ---- 
True
{'startDateIntervalBegin': '2000-01-01T00:00:00', 'startDateIntervalEnd': '2030-12-31T23:59:59', 'showFinishedInstanceTasks': False, 'showPendingInstanceTasks': False, 'active': False, 'flowId': '255'}
---- >> Requisicao forçada... << ---- 


In [132]:
layouts, docs = contagem_layouts_documentos(df, instancesReport)
contagens = contagem_CS1(df) + contagem_CS2(df,instancesReport, instancesReportClose) + contagem_fluxo_personalizado(instancesReport) + docs + layouts
listIndex =['CONTATOS EM ABERTO - CS1', 'CONTATOS EFETIVOS - CS1', 'CONTATOS ATRASADOS - CS1',
            'CONTATOS EM ABERTO - CS2', 'CONTATOS EFETIVOS - CS2', 'CONTATOS ATRASADOS - CS2',
            'Nº DE CLIENTES NO ACOMPANHAMENTO PERSONALIZADO',
            'ENVIOS DE DOCUMENTOS PENDENTES', 'ENVIOS DE DOCUMENTOS REALIZADOS', 'ENVIOS DE DOCUMENTOS ATRASADOS',
            'ENVIOS DE LAYOUT PENDENTE', 'ENVIOS DE LAYOUT REALIZADOS', 'ENVIOS DE LAYOUT ATRASADOS']
contagemDF = pd.DataFrame(contagens, index=listIndex,columns=['Contagem'])
contagemDF

C:\Users\Lukas\AppData\Local\Temp\ipykernel_16016\2754807476.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solicitationsCloseDF['endDateTime'] = pd.to_datetime(solicitationsCloseDF['endDateTime'])
C:\Users\Lukas\AppData\Local\Temp\ipykernel_16016\2754807476.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solicitationsCloseDF['endDateTime'] = solicitationsCloseDF['endDateTime'].dt.strftime('%Y-%m-%d')
C:\Users\Lukas\AppData\Local\Temp\ipykernel_16016\2754807476.py:72: SettingWithCopyWarning: 
A v

,Contagem
CONTATOS EM ABERTO - CS1,70
CONTATOS EFETIVOS - CS1,40
CONTATOS ATRASADOS - CS1,16
CONTATOS EM ABERTO - CS2,84
CONTATOS EFETIVOS - CS2,22
CONTATOS ATRASADOS - CS2,17
Nº DE CLIENTES NO ACOMPANHAMENTO PERSONALIZADO,24
ENVIOS DE DOCUMENTOS PENDENTES,7
ENVIOS DE DOCUMENTOS REALIZADOS,0
ENVIOS DE DOCUMENTOS ATRASADOS,5


In [204]:
def arquivo_Existe(caminhoArquivo):
    try:
        localFile = os.path.exists(caminhoArquivo)
        if os.path.exists(localFile):
            print('Arquivo existe!')
            return localFile, 1
        else:
            print('Arquivo nao existe!')
            return 0, 0
    except Exception as e:
        print('Erro na funcao -> arquivo_Attachment')
        funcoes.printError(e)
        return -1, -1

def salva_Envia_Email(data1:str, data2:str, mensagem:str, enderecosEnvio:list=None):
    try:
        # Nome do arquivo OKrs
        fileName = "OKRs_" + data1 + "_ate_" + data2 + ".xlsx"
        # Cria arquivo xslx e salva
        contagemDF.to_excel("arquivosOkrs/" + fileName)
        # Verifica se criou
        absPath, existe = arquivo_Existe("arquivosOkrs/" + fileName)
        # Se arquivo existe

        # Envia email
        password = 'xeppbwvbemvrpgms'
        app  = Flask(__name__)
        mail = Mail(app)

        app.config['MAIL_SERVER'] = 'smtp.gmail.com'
        app.config['MAIL_PORT'] = 465
        app.config['MAIL_USERNAME'] = 'bot.elysia@gmail.com'
        app.config['MAIL_PASSWORD'] = password
        app.config['MAIL_USE_TLS'] = False
        app.config['MAIL_USE_SSL'] = True
        mail = Mail(app)

        with app.app_context():
            msg = Message(subject="Contagem OKRs Customer Success - " + parser.parse(data1).strftime("%d/%m/%Y") + " até " + parser.parse(data2).strftime("%d/%m/%Y") + ".", sender='bot.elysia@gmail.com', recipients=enderecosEnvio)
            msg.body = mensagem

            # Se for para enviar com attachment
            if existe == 1:
                with app.open_resource("arquivosOkrs/" + fileName) as fp:
                    msg.attach(fileName, "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet", fp.read())

            # Envia Email
            mail.send(msg)
        return 'Enviado com sucesso as {0}.'.format(datetime.today().strftime("%d/%m/%Y %H:%M:%S"))
    except Exception as e:
        funcoes.printError(e)
        return 'Nao foi possivel enviar o email.'

In [208]:
mensagem = 'Segue contagem de OKRs realizada pela automação de OKRs.\nPeríodo de ' + periodo1 + " até " + periodo2 + ".\n"
print(salva_Envia_Email(periodo1,periodo2,mensagem,['lukas.machado@elysia.com.br','teste@elysia.com.br','admin@elysia.com.br']))

Arquivo existe!
Enviado com sucesso as 08/11/2022 17:02:05.
